In [750]:
#Model 1

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor

In [15]:
df = pd.read_csv('weather.csv')

In [16]:
unique_provinces = df['province'].unique()
num_provinces = len(unique_provinces)

# Mã hóa OneHotEncoder cho các tỉnh thành
df_province = pd.DataFrame({'province': unique_provinces})
onehot_encoder = OneHotEncoder(sparse_output=False)
encoded_provinces = onehot_encoder.fit_transform(df_province[['province']])

df_province_encoded = pd.DataFrame(encoded_provinces, columns=onehot_encoder.get_feature_names_out(['province']))

# Thêm các cột mã hóa vào DataFrame gốc
df = pd.concat([df.reset_index(drop=True), df_province_encoded.reset_index(drop=True)], axis=1)

In [ ]:
# Xác định có mưa
df['have_rain'] = df['rain'] > 0

df['have_rain'] = df['have_rain'].astype(bool)

# Kiểm tra phân tích hướng gió
unique_wind_d = df['wind_d'].unique()
num_wind_d = len(unique_wind_d)

# Định dạng hướng gió
wind_direction_mapping = {
    'N': 0,
    'NNE': 22.5,
    'NE': 45,
    'ENE': 67.5,
    'E': 90,
    'ESE': 112.5,
    'SE': 135,
    'SSE': 157.5,
    'S': 180,
    'SSW': 202.5,
    'SW': 225,
    'WSW': 247.5,
    'W': 270,
    'WNW': 292.5,
    'NW': 315,
    'NNW': 337.5
}

df['wind_degree'] = df['wind_d'].map(wind_direction_mapping)

df['date'] = pd.to_datetime(df['date'])

df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['day_of_week'] = df['date'].dt.day_name()  
df['week_of_year'] = df['date'].dt.isocalendar().week 
df['year_quarter'] = df['date'].dt.to_period('Q').astype('int')
df['month_period'] = df['date'].dt.to_period('M').astype('int')

day_of_week_mapping = {
    'Monday': 1,
    'Tuesday': 2,
    'Wednesday': 3,
    'Thursday': 4,
    'Friday': 5,
    'Saturday': 6,
    'Sunday': 7
}

df['day_of_week_encoded'] = df['day_of_week'].map(day_of_week_mapping)


df.info()

In [18]:
# Thay đổi X để sử dụng cột đã mã hóa
X = df[['province', 'year', 'month', 'day', 'day_of_week_encoded', 'week_of_year', 'year_quarter', 'month_period']]
y = df[['max', 'min', 'wind', 'wind_degree', 'humidi', 'cloud', 'pressure']]

In [19]:
# Mã hóa tỉnh thành phố
encoder = OneHotEncoder(sparse_output=False)
X_encoded = encoder.fit_transform(X[['province']])
X_encoded_df = pd.DataFrame(X_encoded, columns=encoder.get_feature_names_out(['province']))

In [20]:
# Kết hợp các cột đã mã hóa với dữ liệu ngày tháng
X_final = pd.concat([X.drop('province', axis=1), X_encoded_df], axis=1)

In [22]:
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size=0.3, random_state=42)

In [23]:
# Tiền xử lý dữ liệu (chuẩn hóa)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [24]:
model = MultiOutputRegressor(RandomForestRegressor(n_estimators=10, random_state=42))

# Huấn luyện mô hình
model.fit(X_train_scaled, y_train)

model1 = model

In [25]:
# Dự đoán trên tập kiểm tra
y_pred = model1.predict(X_test_scaled)

In [30]:
# Đánh giá mô hình
print('Max Temperature - MSE:', mean_squared_error(y_test['max'], y_pred[:, 0]))
print('Max Temperature - R^2:', r2_score(y_test['max'], y_pred[:, 0]))
print('Min Temperature - MSE:', mean_squared_error(y_test['min'], y_pred[:, 1]))
print('Min Temperature - R^2:', r2_score(y_test['min'], y_pred[:, 1]))
print('Wind - MSE:', mean_squared_error(y_test['wind'], y_pred[:, 2]))
print('Wind - R^2:', r2_score(y_test['wind'], y_pred[:, 2]))
print('Wind Degree - MSE:', mean_squared_error(y_test['wind_degree'], y_pred[:, 3]))
print('Wind Degree - R^2:', r2_score(y_test['wind_degree'], y_pred[:, 3]))
print('Humidity - MSE:', mean_squared_error(y_test['humidi'], y_pred[:, 4]))
print('Humidity - R^2:', r2_score(y_test['humidi'], y_pred[:, 4]))
print('Cloud - MSE:', mean_squared_error(y_test['cloud'], y_pred[:, 5]))
print('Cloud - R^2:', r2_score(y_test['cloud'], y_pred[:, 5]))
print('Pressure - MSE:', mean_squared_error(y_test['pressure'], y_pred[:, 6]))
print('Pressure - R^2:', r2_score(y_test['pressure'], y_pred[:, 6]))

Max Temperature - MSE: 3.199297464644244
Max Temperature - R^2: 0.8465952888786461
Min Temperature - MSE: 1.6806495933172123
Min Temperature - R^2: 0.891760083673452
Wind - MSE: 8.797175386531837
Wind - R^2: 0.6861836180988545
Wind Degree - MSE: 1720.2405233293032
Wind Degree - R^2: 0.6595295176498941
Humidity - MSE: 23.75777130504873
Humidity - R^2: 0.7254508239601418
Cloud - MSE: 238.5447365721404
Cloud - R^2: 0.583872036941538
Pressure - MSE: 2.710213416868175
Pressure - R^2: 0.874013525133067


In [ ]:
# Tạo DataFrame cho ngày cụ thể
data = {
    'province': ['Ho Chi Minh City'],
    'date': [pd.to_datetime('2024-09-20')],
}

df_input = pd.DataFrame(data)


# Tiền xử lý dữ liệu đầu vào
df_input['year'] = df_input['date'].dt.year
df_input['month'] = df_input['date'].dt.month
df_input['day'] = df_input['date'].dt.day
df_input['day_of_week'] = df_input['date'].dt.day_name()
df_input['week_of_year'] = df_input['date'].dt.isocalendar().week
df_input['year_quarter'] = df_input['date'].dt.to_period('Q').astype('int')
df_input['month_period'] = df_input['date'].dt.to_period('M').astype('int')

day_of_week_mapping = {
    'Monday': 1,
    'Tuesday': 2,
    'Wednesday': 3,
    'Thursday': 4,
    'Friday': 5,
    'Saturday': 6,
    'Sunday': 7
}

df_input['day_of_week_encoded'] = df_input['day_of_week'].map(day_of_week_mapping)

# Mã hóa tỉnh thành
province_encoded = encoder.transform(df_input[['province']])
province_encoded_df = pd.DataFrame(province_encoded, columns=encoder.get_feature_names_out(['province']))

# Kết hợp các cột đã mã hóa với dữ liệu ngày tháng
X_input = pd.concat([df_input.drop(['province', 'date', 'day_of_week'], axis=1), province_encoded_df], axis=1)

# Đảm bảo thứ tự cột đúng bằng cách sắp xếp lại theo thứ tự của X_train
X_input = X_input.reindex(columns=X_final.columns, fill_value=0)

# Chuẩn hóa dữ liệu đầu vào
X_input_scaled = scaler.transform(X_input)

# Dự đoán
y_pred_input = model.predict(X_input_scaled)

# In kết quả dự đoán
print('Dự đoán cho Ho Chi Minh City vào ngày 2024-09-20:')
print('Max Temperature:', y_pred_input[0, 0])
print('Min Temperature:', y_pred_input[0, 1])
print('Wind Speed:', y_pred_input[0, 2])
print('Wind Degree:', y_pred_input[0, 3])
print('Humidity:', y_pred_input[0, 4])
print('Cloud:', y_pred_input[0, 5])
print('Pressure:', y_pred_input[0, 6])


In [ ]:
import joblib

joblib.dump(model1, 'weather_model_r.pkl')